## Generate synthetic data [needs work in course selection algortihm]
## convert text to numeric using tfidf [not used currently]

In [240]:
import pandas as pd
pd.set_option('display.max_columns', None) 
from os import listdir
import os
import glob
import spacy
from collections import Counter
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
from spacy.training import Example
from spacy.util import minibatch, compounding
from sklearn.model_selection import train_test_split
from spacy.scorer import Scorer
import re
import nlp
#https://chat.openai.com/share/c141bbaf-9fa9-41df-b29a-68653cfafd48

[nltk_data] Downloading package stopwords to /Users/nyzy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [241]:
df_c_cleaned = pd.read_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/course_cleaned.csv")
df_c_cleaned=df_c_cleaned.rename(columns={"Unnamed: 0":"course_number"})
df_j_cleaned=pd.read_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/job_cleaned_salary.csv")
df_s_skills_list = pd.read_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/skills_dataset/skills_list_df.csv")

skills_list = df_s_skills_list["skills_list"].unique().tolist()

In [242]:
#filtering unmatched rows
df_j_cleaned = df_j_cleaned[df_j_cleaned["title_cleaned"]!="unmatched"]
#df_j_cleaned[df_j_cleaned["title_cleaned"]=="unmatched"].to_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/job_title_unmatched.csv")


In [243]:
# Loading the JSON file back into a dictionary
import json
Job_progression_dictionary_file_path = "/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/job_progression_dictionary.json"

with open(Job_progression_dictionary_file_path, 'r') as json_file:
    sorted_full_job_progression_dict_lower = json.load(json_file)

print(sorted_full_job_progression_dict_lower)
# Output: {'name': 'John', 'age': 30, 'city': 'New York'}


{'ai architect': ['ai project manager', 'head of ai', 'chief ai officer'], 'ai developer': ['senior ai developer', 'ai architect', 'ai project manager', 'head of ai', 'chief ai officer'], 'ai project manager': ['head of ai', 'chief ai officer'], 'big data architect': ['head of big data', 'director of data engineering', 'chief information officer'], 'big data engineer': ['senior big data engineer', 'big data architect', 'head of big data', 'director of data engineering', 'chief information officer'], 'business analysis manager': ['business intelligence analyst', 'director of business analysis', 'chief strategy officer', 'data analyst', 'data scientist'], 'business analyst': ['senior business analyst', 'business analysis manager', 'business intelligence analyst', 'director of business analysis', 'chief strategy officer', 'data analyst', 'data scientist'], 'business intelligence analyst': ['senior business intelligence analyst', 'business intelligence manager', 'director of business intel

In [ ]:
import pandas as pd
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ast
# Assuming you have the following data loaded into pandas DataFrames
# jobs_df = pd.DataFrame({'Job': ['Data Analyst', 'Data Scientist', ...], 'Skills': [['SQL', 'Python'], ['Machine Learning', 'Python', 'SQL'], ...]})
# courses_df = pd.DataFrame({'Course': ['Intro to Python', 'Advanced Machine Learning', ...], 'Skills_Taught': [['Python'], ['Machine Learning', 'Deep Learning'], ...]})
# skills_df = pd.DataFrame({'Skill': ['Python', 'SQL', 'Machine Learning', ...]})
# A simple function to select a random job and its required skills


def select_random_job(jobs_df):
    random_idx = random.randint(0, len(jobs_df) - 1)
    random_job = jobs_df.iloc[random_idx]
    return random_job.title_cleaned, random_job.skills_tagged,random_job.normalized_min,random_job.normalized_max


def select_next_job(current_job, jobs_df, set_job_unique,job_progression_dict):
    next_job,next_skills,random_job_selection_flag="","",0
    # If the current job is in the progression dict, select the next job from there
    if current_job in job_progression_dict:
        next_jobs = job_progression_dict[current_job]
        next_jobs=list(set_job_unique.intersection(set(next_jobs)))
        if len(next_jobs)!=0:
            next_job = random.choice(next_jobs)
            #print(next_job, " choosen")
            try:
                #next_skills = jobs_df[jobs_df['title_cleaned'] == next_job]['skills_tagged'].values[0]
                next_skills_rows = jobs_df[jobs_df['title_cleaned'] == next_job]
                if not next_skills_rows.empty:
                    random_row = next_skills_rows.sample(n=1)
                    next_skills = random_row['skills_tagged'].values[0]
            except:
                next_job = random.choice(next_jobs)
                #next_skills = jobs_df[jobs_df['title_cleaned'] == next_job]['skills_tagged'].values[0]
                next_skills_rows = jobs_df[jobs_df['title_cleaned'] == next_job]
                if not next_skills_rows.empty:
                    random_row = next_skills_rows.sample(n=1)
                    next_skills = random_row['skills_tagged'].values[0]
        else:
            # If not, fall back to random selection for this example
            next_job, next_skills,_,_ = select_random_job(jobs_df)
            random_job_selection_flag=1
    else:
        next_job, next_skills,_,_ = select_random_job(jobs_df)
        random_job_selection_flag=1
    return next_job, next_skills,random_job_selection_flag





# A simple function to select a course that teaches at least one of the required skills
def select_relevant_course(courses_df, required_skills):
    relevant_courses = courses_df[courses_df['Skills_Taught'].apply(lambda x: any(skill in x for skill in required_skills))]
    if not relevant_courses.empty:
        return relevant_courses.sample()
    return None


def calculate_course_score(course, skill_gap, weight_skill_match=1, weight_subscribers=0.001, weight_rating=0.5):
    # Calculate the proportion of skill gap covered by course
    skills_matched = sum(skill in course['skills_tagged'] for skill in skill_gap)
    skill_match_score = skills_matched / len(skill_gap) if skill_gap else 0

    # Factor in course popularity and rating
    #subscriber_score = course['num_subscribers'] * weight_subscribers if 'num_subscribers' in course else 0
    #rating_score = course['rating'] * weight_rating if 'rating' in course else 0

    # Calculate total score
    total_score = (skill_match_score * weight_skill_match) #+ subscriber_score + rating_score
    return total_score

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_c_cleaned["full_description"])

def calculate_similarity_score(skill_gap,df_c_cleaned):

    def preprocess_text(text):
        return " ".join(token.lemma_.lower() for token in nlp(text) if not token.is_punct and not token.is_stop)

    skill_gap_text = " ".join(skill_gap)
    # Vectorize the text

    skill_gap_vec = tfidf_vectorizer.transform([skill_gap_text])
    # Calculate cosine similarity
    cosine_similarities = cosine_similarity(skill_gap_vec, tfidf_matrix).flatten()
    df_c_cleaned["cosine_score"]=cosine_similarities
    return df_c_cleaned

def select_relevant_course(df_c_cleaned, skill_gap):
    # Apply the scoring function to each course
    df_c_cleaned['course_score'] = df_c_cleaned.apply(calculate_course_score, axis=1, skill_gap=skill_gap)
    df_c_cleaned = calculate_similarity_score(skill_gap,df_c_cleaned)
    # Sort courses based on the calculated score
    #df_c_cleaned = df_c_cleaned.sort_values(by='course_score', ascending=False)
    df_c_cleaned = df_c_cleaned.sort_values(by='cosine_score', ascending=False)
    # Select the top course if available
    top_course = df_c_cleaned.iloc[0] if not df_c_cleaned.empty else None
    top_10_course_number = df_c_cleaned["course_number"].iloc[0:10].tolist()
    top_10_course_number_score = {x:round(y,3) for x,y in zip(df_c_cleaned["course_number"].iloc[0:10].tolist(),df_c_cleaned["cosine_score"].iloc[0:10].tolist())}
    return top_course,top_10_course_number_score,top_10_course_number

# Select the most relevant course based on the skill gap
#relevant_course = select_relevant_course(df_c_cleaned, skill_gap)
#relevant_course.head()

# Generate synthetic user data
def generate_synthetic_data(jobs_df, courses_df, skills_df, num_samples=10):
    synthetic_data = []
    set_job_unique = set(jobs_df["title_cleaned"].unique())
    for _ in range(num_samples):
        #select_random_job(jobs_df)
        current_job, current_skills,current_min_salary,current_max_salary = select_random_job(jobs_df)
        # Select a job to progress to that requires at least one additional skill
        next_job, next_skills,random_job_selection_flag = select_next_job(current_job,jobs_df,set_job_unique,sorted_full_job_progression_dict_lower)
        #while set(next_skills).issubset(current_skills):
        #    next_job, next_skills = select_random_job(jobs_df)
        next_skills=ast.literal_eval(next_skills)
        current_skills=ast.literal_eval(current_skills)
        # Find the skill gap
        skill_gap = list(set(next_skills) - set(current_skills))
        #print(skill_gap)
        if  len(skill_gap)==0:
            skill_gap = list(next_skills)
        # Find a course that teaches one of the skills in the gap
        course,top_10_courses_score,top_10_courses_number = select_relevant_course(courses_df, skill_gap)
        #print(course)
        if course is not None:
            course_name = course.course_title
            course_skills = course.skills_tagged
            course_title = course.title
        else:
            # If no single course covers the gap, select a random one for this synthetic example
            course_name = "Random Course"
            course_skills = "Random Skill"

        if len(current_skills) and len(next_skills):
            # Append to the synthetic data list
            synthetic_data.append({
                'Current Job': current_job,
                'Next Job': next_job,
                'Current Skills': current_skills,
                'Next Skills': next_skills,
                'Skill Gap': skill_gap,
                'Current Min Salary':current_min_salary,
                'Current Max Salary':current_max_salary,
                'course title':course_title,
                'Course Taken': course_name,
                'Course Skills': course_skills,
                'top_10_course_numbers':top_10_courses_number,
                'top_10_course_score':top_10_courses_score,
                'random next job':random_job_selection_flag
            })
    
    
    return pd.DataFrame(synthetic_data)


# Generate synthetic user data
def generate_synthetic_next_job_data(jobs_df,courses_df,synthetic_user_data):
    synthetic_data = []
    set_job_unique = set(jobs_df["title_cleaned"].unique())
    for i,iterrow in synthetic_user_data.iterrows():
        #print(i, iterrow["Next Job"],iterrow["Next Skills"])
        #select_random_job(jobs_df)
        current_job, current_skills,current_min_salary,current_max_salary = iterrow["Next Job"],(iterrow["Next Skills"]),iterrow["Current Min Salary"],iterrow["Current Max Salary"]
        # Select a job to progress to that requires at least one additional skill
        next_job, next_skills,random_job_selection_flag = select_next_job(current_job,jobs_df,set_job_unique,sorted_full_job_progression_dict_lower)
        #while set(next_skills).issubset(current_skills):
        #    next_job, next_skills = select_random_job(jobs_df)
        next_skills=ast.literal_eval(next_skills)
        # Find the skill gap
        skill_gap = list(set(next_skills) - set(current_skills))
        #print(skill_gap)
        # Find a course that teaches one of the skills in the gap
        course,top_10_courses_score,top_10_courses_number = select_relevant_course(courses_df, skill_gap)
        #print(course)
        if course is not None:
            course_name = course.course_title
            course_skills = course.skills_tagged
            course_title = course.title
        else:
            # If no single course covers the gap, select a random one for this synthetic example
            course_name = "Random Course"
            course_skills = "Random Skill"
        
        if len(current_skills) and len(next_skills):
            # Append to the synthetic data list
            synthetic_data.append({
                'Current Job': current_job,
                'Next Job': next_job,
                'Current Skills': current_skills,
                'Next Skills': next_skills,
                'Skill Gap': skill_gap,
                'Current Min Salary':current_min_salary,
                'Current Max Salary':current_max_salary,
                'course title':course_title,
                'Course Taken': course_name,
                'Course Skills': course_skills,
                'top_10_course_numbers':top_10_courses_number,
                'top_10_course_score':top_10_courses_score,
                'random next job':random_job_selection_flag
            })
    
    return pd.DataFrame(synthetic_data)



# Generate the synthetic data
synthetic_user_data_v1 = generate_synthetic_data(df_j_cleaned, df_c_cleaned, skills_list,25000)
synthetic_user__next_data_v2 = generate_synthetic_next_job_data(df_j_cleaned, df_c_cleaned,synthetic_user_data_v1)
synthetic_user_data = pd.concat([synthetic_user_data_v1,synthetic_user__next_data_v2],axis=0)
#synthetic_user__next_data_v3 = generate_synthetic_next_job_data(df_j_cleaned, df_c_cleaned,synthetic_user__next_data_v2)
#synthetic_user__next_data_v4 = generate_synthetic_next_job_data(df_j_cleaned, df_c_cleaned,synthetic_user__next_data_v3)
#synthetic_user_data = pd.concat([synthetic_user_data_v1,synthetic_user__next_data_v2,synthetic_user__next_data_v3,synthetic_user__next_data_v4],axis=0)
#print(synthetic_user_data.head())


In [225]:

synthetic_user_data.to_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/synthetic_data_half_cleaned.csv")



,Current Job,Next Job,Current Skills,Next Skills,Skill Gap,Current Min Salary,Current Max Salary,course title,Course Taken,Course Skills,top_10_course_numbers,top_10_course_score,random next job
0,director of data engineering,chief technology officer,"{computer science, test data management, devops}",{business requirements},[business requirements],-0.712324,-0.711278,data science business context,data analysis,"{'python', 'data science', 'probability'}","{1696: 0.184, 1984: 0.184, 59: 0.176, 119: 0.1...","[1696, 1984, 59, 119, 1904, 1021, 1690, 1832, ...",0
1,chief data officer,director of analytics,"{analytics, computer science, generative ai, b...","{analytics, sql, data retrieval, data warehous...","[sql, data retrieval, data warehousing, algori...",-0.543054,-0.616494,new methods data visualization python,data visualization,"{'data visualization', 'data science', 'visual...","{943: 0.273, 2914: 0.269, 755: 0.266, 2135: 0....","[943, 2914, 755, 2135, 1794, 2735, 1445, 1357,...",1
2,data science manager,chief data scientist,"{analytics, problem solving, sql, machine lear...","{data processing, algorithms, big data, comput...","[algorithms, data processing, machine learning...",-0.292600,-0.334121,big data programming languages big data vs dat...,big data,"{'data science', 'big data'}","{289: 0.441, 827: 0.417, 1043: 0.384, 2099: 0....","[289, 827, 1043, 2099, 137, 1525, 372, 720, 84...",0
3,data analyst,big data architect,"{analytics, pyspark, data mining, probability,...","{analytics, problem solving, sql, data securit...","[problem solving, data security, business anal...",-1.313989,-1.379718,data science fundamentals,data science,"{'computer science', 'data science', 'algorith...","{1520: 0.394, 289: 0.386, 1043: 0.379, 841: 0....","[1520, 289, 1043, 841, 2036, 323, 968, 1521, 1...",0
4,senior data scientist,director of data science,"{data wrangling, sql, visualization, python, w...","{data analysis, analytics, data science, gener...","[data analysis, analytics, generative ai, dist...",0.292525,0.284992,statistics statistical modeling explained python,data analysis,"{'data science', 'algorithms', 'test statistic...","{1461: 0.355, 1218: 0.351, 1478: 0.349, 2036: ...","[1461, 1218, 1478, 2036, 1711, 1747, 172, 1973...",0
5,chief product officer,chief information officer,"{project management, product analysis}","{project management, microsoft office, busines...","[microsoft office, business development, sql]",-0.005407,-0.010397,complete microsoft sql server scratch bootcamp,data science,"{'data science', 'business intelligence', 'sql...","{962: 0.391, 2914: 0.39, 2735: 0.371, 2600: 0....","[962, 2914, 2735, 2600, 701, 1686, 2865, 315, ...",0
6,big data architect,chief information officer,{big data},"{computer science, data processing, azure acti...","[computer science, data processing, azure acti...",1.463122,1.445596,python sdk azure bootcamp,undefined,"{'azure data lake', 'big data', 'cloud computi...","{1178: 0.36, 2089: 0.328, 768: 0.3, 2030: 0.29...","[1178, 2089, 768, 2030, 332, 1806, 2020, 832, ...",0
7,data architect,director of analytics,"{data dictionary, http, agile software develop...","{operations analytics, microsoft office}","[microsoft office, operations analytics]",-1.320774,-1.404756,avaya ip office server,artificial intelligence,{'artificial intelligence'},"{701: 0.32, 1786: 0.25, 569: 0.209, 1883: 0.16...","[701, 1786, 569, 1883, 2546, 1992, 1182, 1784,...",0
8,data science manager,chief data scientist,"{analytics, data engineering, sql, project man...","{data processing, algorithms, big data, comput...","[algorithms, data processing, machine learning...",-0.919509,-0.916695,big data programming languages big data vs dat...,big data,"{'data science', 'big data'}","{289: 0.441, 827: 0.417, 1043: 0.384, 2099: 0....","[289, 827, 1043, 2099, 137, 1525, 372, 720, 84...",0


In [234]:
import ast 
df = synthetic_user_data.copy()
# Fill missing values if necessary
df.fillna({'Course Taken': 'no course','course title': 'no course'}, inplace=True)
df["Course Taken"]=df["Course Taken"].apply(lambda x:x.replace("NAN","No Course"))

df["Course Skills"]=df["Course Skills"].apply(lambda x:ast.literal_eval(x))
df["Course Skills"].apply(lambda x: x if len(x)!=0 else "no skills")
#df["Course title"]=df["Course title"].apply(lambda x:x.replace("NAN","No Course"))
def set_to_string(skill_set):
    if not skill_set:  # Checks if the set is empty
        return 'no skills' 
    # Check if skill_set is a string that needs to be evaluated
    if isinstance(skill_set, str):
        try:
            # Try to evaluate the string as a set
            skill_set = ast.literal_eval(skill_set)
        except (ValueError, SyntaxError):
            # Handle cases where the string is not a valid set
            pass  # You might want to return a default value or handle this case as needed
    # Convert to string if it's a set or list
    if isinstance(skill_set, (set, list)):
        return ', '.join(skill_set)
    return skill_set 

set_columns = ['Current Skills', 'Next Skills', 'Skill Gap', 'Course Skills']
for col in set_columns:
    print(col)
    #df[col]=df[col].apply(lambda x:ast.literal_eval(x))
    df[col] = df[col].apply(set_to_string)





Current Skills
Next Skills
Skill Gap
Course Skills


In [63]:
df.to_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/synthetic_data_cleaned_v3.csv")


In [14]:
# convert text to numeric using tfidf [not used currently]

import spacy

# Load SpaCy model
nlp = spacy.load('en_core_web_md')

def preprocess_text(text):
    doc = nlp(text)
    return " ".join(token.lower_ for token in doc if not token.is_punct and not token.is_stop)

text_columns = ['Current Job', 'Next Job', 'Current Skills', 'Next Skills', 'Skill Gap', 'course title', 'Course Skills','Course Taken']
for col in text_columns:
    print(col)
    #df[col] = df[col].apply(preprocess_text)

from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Assuming df is your DataFrame and text_columns are the columns to vectorize
tfidf_vectorizer = TfidfVectorizer()

# Create a DataFrame to hold all TF-IDF features
tfidf_df = pd.DataFrame()

for col in ["course title"]:
    # Apply TF-IDF transformation
    tfidf_matrix = tfidf_vectorizer.fit_transform(df[col])

    # Convert to dense format and create a DataFrame
    tfidf_dense = pd.DataFrame(tfidf_matrix.todense(), columns=tfidf_vectorizer.get_feature_names_out())

    # Prefix column names with the original column name for clarity
    tfidf_dense.columns = [f"{col}_{feature}" for feature in tfidf_dense.columns]

    # Concatenate with the main TF-IDF DataFrame
    tfidf_df = pd.concat([tfidf_df, tfidf_dense], axis=1)

# Combine the original DataFrame with the TF-IDF features DataFrame
final_df = pd.concat([df, tfidf_df], axis=1)

Current Job
Next Job
Current Skills
Next Skills
Skill Gap
course title
Course Skills
Course Taken


In [15]:
final_df

,Current Job,Next Job,Current Skills,Next Skills,Skill Gap,course title,Course Taken,Course Skills,random next job,course title_ai,course title_bonus,course title_chatgpt,course title_learning,course title_machine,course title_python
0,ai project manager,data scientist,"project management, problem solving, microsoft...","analytics, sql, python, algorithms, data stora...","d, h, u, q, y",machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",1,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
1,product analyst,data product manager,,"project management, analytics, agile software ...","w, ', v, o, }, {, , g, b, c, a, j, m, ,, p, d...",machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",0,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
2,data analyst,director of data engineering,"microsoft office, data synchronization, data m...","computer science, test data management, devops","u, v",machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",0,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
3,unmatched,database administrator,"problem solving, data collection","database administration, sql",q,machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",1,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
4,senior of data science,statistician,"data analysis, analytics, business intelligenc...","data analysis, statistical analysis, data mana...",no skills,machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",1,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
5,unmatched,machine learning engineer,"problem solving, project management, data mana...","data mining, bigtable, mapreduce, python, comp...","h, u, f",machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",1,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
6,director of product management,chief product officer,"technology solutions, problem solving, risk an...","analytics, data driven decision making, produc...","d, f",machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",0,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
7,business analyst,business intelligence analyst,"business analysis, qualitative analysis, sql","analytics, business intelligence","g, c",machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",0,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
8,business intelligence manager,director of business intelligence,"data analysis, analytics, problem solving, bus...","data analysis, analytics, business intelligenc...","k, f, w",machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",0,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
9,director of analytics,unmatched,"analytics, data science, computer science, com...","data structures, computer science, data integr...",k,machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",1,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
